In [6]:
from web3 import Web3
import requests
import json
from pprint import pprint

# Connect to Ethereum node
web3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/58ea22f2caa14187bd2b8c0682c84848'))

ETHERSCAN_API_KEY = "6AMB9PGBYJ5AHHCZHCCZAU5Y7E4KEVET47"

In [2]:
############################### Start definitions ###############################

class TransactionHandler:
    def __init__(self):
        pass

    def get_contract_name(self, contract_address: str) -> str or None:
        """
        Arguements:
            contract_address: contract address of smart contract to find name of
        Returns:
            String value of contracts name if available OR None if name is not available
        Notes:
            Calls Etherscan's source code API endpoint to find the smart contract's name
        """
        # Get contract name from source code
        source_code_endpoint = f"https://api.etherscan.io/api?module=contract&action=getsourcecode&address={contract_address}&apikey={ETHERSCAN_API_KEY}"
        source_code = json.loads(requests.get(source_code_endpoint).text)
        try:
            return source_code["result"][0]["ContractName"]
        except (KeyError, TypeError):
            return None

    def get_transaction_type(self, tx: str or dict) -> str:
        """
        Arguements:
            tx: Tx hash (str) OR Tx object (dict)
        Returns:
            String value representing type of transaction
        """
        # If 'tx' arguement is a transaction hash, get transaction object
        if isinstance(tx, str):
            tx = web3.eth.get_transaction(tx)
        # If "input" is not empty ('0x0'), "to" is not empty and "value" is equal to 0 -> smart contract transaction
        if tx["input"] != "0x" and tx["to"] != None and tx["value"] == 0:
            return "contract_invoked"
        # If "input" is empty ('0x') and "value" is not equal to 0 -> wallet-to-wallet transaction
        elif tx["input"] == "0x" and tx["value"] != 0:
            return "wallet_to_wallet"
        # If "to" is empty and "value" is equal to 0 -> contract creation transaction
        elif tx["to"] == None and tx["value"] == 0:
            return "contract_creation"
        else:
            raise RuntimeError(f"Unknown transaction type found for {tx['hash']}")

    def decode_transaction_input(self, tx: str or dict) -> dict:
        """
        Arguements:
            tx_input: Raw input data from transaction. 
            contract_address: Smart contract address. Use the "to" address on an Ethereum transaction.
        Returns:
            Dictionary containing the smart contract function's name, arguments and transaction parameters.
        Notes:
            Can only decode transactions with transaction type 'contract_invoked' returned by get_transaction_type()
        """
        # If 'tx' arguement is a transaction hash, get transaction object
        if isinstance(tx, str):
            tx = web3.eth.get_transaction(tx)
        # Check for supported transaction type
        if self.get_transaction_type(tx) != "contract_invoked":
            raise RuntimeError(f"Cannot parse input of transaction with type '{self.get_transaction_type(tx)}'.")
        # Get ABI for smart contract NOTE: Use "to" address as smart contract 'interacted with'
        abi_endpoint = f"https://api.etherscan.io/api?module=contract&action=getabi&address={tx['to']}&apikey={ETHERSCAN_API_KEY}"
        abi = json.loads(requests.get(abi_endpoint).text)
        # If Etherscan API returns 'Contract source code not verified', raise error
        if abi["result"] == "Contract source code not verified":
            raise RuntimeError(f"{abi['result']}. Recommended to hard-code the ABI of this smart contract address {tx['to']}.")
        # Create contract object in web3
        contract = web3.eth.contract(address=tx["to"], abi=abi["result"])
        # Decode transaction input data using Contract object's decode_function_input() method
        func_obj, func_params = contract.decode_function_input(tx["input"])
        return {
            "func_name": func_obj.fn_name,
            "args": tuple(func_params.keys()),
            "params": tuple(func_params.values()),
        }

############################### End definitions ###############################

In [4]:
# Tx hash's of different tx types
token_swap = "0xac80bab0940f061e184b0dda380d994e6fc14ab5d0c6f689035631c81bfe220b" # "value" == 0, "to"/"from"/"input" are not empty
wallet_to_wallet = "0xc46bfff90f8a55a1037f62ab3183bc840b3e7eff6f84a0b6e142f849ee4b20be" # "input" is empty ('0x'), "value" != 0
contract_creation = "0x4ff39eceee7ba9a63736eae38be69b10347975ff5fa4d9b85743a51e1c384094" # "to" is empty ('0x0' or 'None'), "value" == 0, "input" is bytecode

# Instantiate transaction handler
th = TransactionHandler()

In [8]:
tx = web3.eth.get_transaction(token_swap)
tx

AttributeDict({'blockHash': HexBytes('0xa495e3d87cf01430e9838341e1d36929fc9c8dfdb89133b372cae4019b5a534a'),
 'blockNumber': 12674755,
 'from': '0x43CC25B1fB6435d8d893fCf308de5C300a568BE2',
 'gas': 1402911,
 'gasPrice': 122000000000,
 'hash': HexBytes('0xac80bab0940f061e184b0dda380d994e6fc14ab5d0c6f689035631c81bfe220b'),
 'input': '0x8803dbee000000000000000000000000000000000000000000000001158e460913d000000000000000000000000000000000000000000000000000003a4837fc5242c4ec00000000000000000000000000000000000000000000000000000000000000a00000000000000000000000009a7ed54b8c2c5816c1800476f5111a1e886575020000000000000000000000000000000000000000000000000000000060cfee3f0000000000000000000000000000000000000000000000000000000000000002000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc20000000000000000000000003c9d6c1c73b31c837832c72e04d3152f051fc1a9',
 'nonce': 11974,
 'r': HexBytes('0x649acf223ea61e4598280adc086f9a3d8a96e7cfc941f25c38d4bf29c3f77484'),
 's': HexBytes('0x66fb1ee9861d493481005

In [10]:
# Get transaction receipt
receipt = web3.eth.get_transaction_receipt(token_swap)
receipt

AttributeDict({'blockHash': HexBytes('0xa495e3d87cf01430e9838341e1d36929fc9c8dfdb89133b372cae4019b5a534a'),
 'blockNumber': 12674755,
 'contractAddress': None,
 'cumulativeGasUsed': 121865,
 'from': '0x43CC25B1fB6435d8d893fCf308de5C300a568BE2',
 'gasUsed': 121865,
 'logs': [AttributeDict({'address': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',
   'blockHash': HexBytes('0xa495e3d87cf01430e9838341e1d36929fc9c8dfdb89133b372cae4019b5a534a'),
   'blockNumber': 12674755,
   'data': '0x0000000000000000000000000000000000000000000000003a395039b420f1bd',
   'logIndex': 0,
   'removed': False,
   'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'),
    HexBytes('0x00000000000000000000000043cc25b1fb6435d8d893fcf308de5c300a568be2'),
    HexBytes('0x00000000000000000000000044d34985826578e5ba24ec78c93be968549bb918')],
   'transactionHash': HexBytes('0xac80bab0940f061e184b0dda380d994e6fc14ab5d0c6f689035631c81bfe220b'),
   'transactionIndex': 0}),
  AttributeDict(

In [9]:
# Get raw block
block = web3.eth.get_block("latest")
block

AttributeDict({'difficulty': 6757105048576577,
 'extraData': HexBytes('0x426162656c20687a38'),
 'gasLimit': 15014632,
 'gasUsed': 15010393,
 'hash': HexBytes('0x220c6c1f55e7f8bbe7c267eba96a3043b401cf40c2730b326629f2c2888c568c'),
 'logsBloom': HexBytes('0xf3e3ea46c147dc095aac7fc48de72a1911a2941f93b954163053358924dc9c3ef6bc13131a0ab6d7c5787b5fc622c929b7a1f0f009077c191664d2e5c577eee570a860220e94ed3be8c2860f49f061f5212001171944102778437cfbe7ef71a6d66fe0868bbf9cc40a2c9cba456ba8706060c45a1d1604764a78e1be340c15205971976bbf6cfcfccc51605d8ddc104e96e49c8d01f2b9098c2665dd2d31c1d482df658112d4b5f20405729cd21336a05a94136b216a10e4e1a60677be7c9e54d0fc84efa86ce1e1212d4cb8c4f9dd9b68e85633bfcba952882170c2156ef960143aa1294a08408528051cbd8197e41ae49c6666627d93f0471e28956dc3d8b6'),
 'miner': '0xB3b7874F13387D44a3398D298B075B7A3505D8d4',
 'mixHash': HexBytes('0x23ea57e01a3da8f70e5334ee5970d09018cc6c4a162afce080bd2dc08a209d1f'),
 'nonce': HexBytes('0x593f90c45a1576d0'),
 'number': 12677940,
 'parentHash': Hex